In [2]:
using BenchmarkTools, Logging, Revise
global_logger(ConsoleLogger(stdout, Logging.Info));

In [3]:
using PorousMaterials, MOFun

[ Info: Precompiling MOFun [top-level]


In [4]:
s_moty = moiety("find-replace/2-!-p-phenylene")
r_moty = moiety("2-acetylamido-p-phenylene")
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent;

# single random replacement w/o search kwarg

In [5]:
@btime find_replace(s_moty, r_moty, parent);

  2.512 s (13347700 allocations: 3.58 GiB)


# single random replacement w/ search kwarg

In [6]:
@btime find_replace(s_moty, r_moty, parent, search=search);

  934.000 μs (23604 allocations: 1.60 MiB)


# single specific replacement

In [7]:
@btime find_replace(s_moty, r_moty, parent, search=search, config=Configuration(1,1));

  933.901 μs (23605 allocations: 1.60 MiB)


# list of (all) replacements
## this runs 40,000-fold faster after refactoring!
### I could functionalize 70,000 MOFs with 20 R-groups in a day on one HPC node!!
#### note to self: write a distributed version...

In [8]:
configs = [Configuration(loc, rand(1:search.num_orientations[loc])) for loc in 1:search.num_locations]
@btime find_replace(s_moty, r_moty, parent, search=search, configs=configs);

  2.957 ms (58125 allocations: 3.63 MiB)


# save output

In [9]:
xtal = find_replace(s_moty, r_moty, parent, search=search, configs=configs)
write_cif(xtal, "OUTPUT")

In [10]:
?Search

search: Search search SearchQuery SearchResult searchsorted searchsortedlast



No documentation found.

# Summary

```
struct Search <: Any
```

# Fields

```
results          :: Array{SearchResult,N} where N
num_isomorphisms :: Int64
num_locations    :: Int64
num_orientations :: Array{Int64,N} where N
```


In [11]:
?SearchResult

search: SearchResult searchsortedlast searchsortedfirst



No documentation found.

# Summary

```
struct SearchResult <: Any
```

# Fields

```
query         :: SearchQuery
configuration :: Configuration
isomorphism   :: Array{Int64,N} where N
```


In [12]:
?Configuration

search: Configuration



No documentation found.

# Summary

```
struct Configuration <: Any
```

# Fields

```
location    :: Int64
orientation :: Int64
```


In [13]:
?SearchQuery

search: SearchQuery



No documentation found.

# Summary

```
struct SearchQuery <: Any
```

# Fields

```
substructure     :: Crystal
parent_structure :: Crystal
```
